# Easy

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/merc.csv')

In [3]:
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,SLK,2005,5200,Automatic,63000,Petrol,325,32.1,1.8
1,S Class,2017,34948,Automatic,27000,Hybrid,20,61.4,2.1
2,SL CLASS,2016,49948,Automatic,6200,Petrol,555,28.0,5.5
3,G Class,2016,61948,Automatic,16000,Petrol,325,30.4,4.0
4,G Class,2016,73948,Automatic,4000,Petrol,325,30.1,4.0


In [11]:
from sklearn.model_selection import train_test_split
x = data.drop(columns=["model", "fuelType", "price", "transmission"])
y = data["price"]
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [12]:
from sklearn.ensemble import RandomForestRegressor

In [13]:
model = RandomForestRegressor(n_estimators=500)

In [14]:
model.fit(x_train, y_train)

RandomForestRegressor(n_estimators=500)

In [16]:
from sklearn.metrics import r2_score

print(r2_score(model.predict(x_test), y_test))

0.8942989351345878


In [18]:
importance = pd.DataFrame(zip(x_train.columns, model.feature_importances_), columns=['Feature', 'Importance'])
print(importance)

      Feature  Importance
0        year    0.179728
1     mileage    0.210725
2         tax    0.032897
3         mpg    0.307105
4  engineSize    0.269545


Для покупателей мерседесов наиболее актуальными показателями являются: потребление топлива и объем бака, менее важны пробег и год производства машины, налог почти не имеет значения

# Medium

In [137]:
import os

# Создаем пустой датафрейм для хранения объединенных данных
combined_data = pd.DataFrame()

# Получаем список файлов в папке "data"
files = [file for file in os.listdir('../data') if file.endswith('.csv')]
print(files)

# Читаем данные из каждого файла и объединяем их в один датафрейм
for file in files:
    data = pd.read_csv(os.path.join('../data', file))
    combined_data = pd.concat([combined_data, data])
    combined_data['price'] = combined_data['price'].replace(',', '.')
    combined_data['mileage'] = combined_data['mileage'].replace(',', '.')
    combined_data['year'] = combined_data['year'].replace(',', '.')
# Выводим первые строки объединенного датафрейма
print(combined_data.head())

['vauxhall.csv', 'bmw.csv', 'vw.csv', 'hyundi.csv', 'audi.csv', 'toyota.csv', 'ford.csv', 'focus.csv', 'skoda.csv', 'cclass.csv', 'merc.csv']
    model  year  price transmission  mileage fuelType    tax   mpg  \
0   Corsa  2018   7885       Manual     9876   Petrol  145.0  55.4   
1   Corsa  2019  11995       Manual     2500   Petrol  145.0  54.3   
2   Corsa  2017   9777    Automatic     9625   Petrol  145.0  47.9   
3   Corsa  2016   8500       Manual    25796   Petrol   30.0  55.4   
4   Corsa  2019  10000       Manual     3887   Petrol  145.0  43.5   

   engineSize  tax(£)  
0         1.4     NaN  
1         1.4     NaN  
2         1.4     NaN  
3         1.4     NaN  
4         1.4     NaN  


In [138]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

combined_data = pd.get_dummies(data=combined_data, columns=["transmission", "fuelType", "model"], drop_first=True)
print(combined_data.head())

x = combined_data.drop(columns=['price'])
y = combined_data['price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.fillna(0, inplace=True)
x_test.fillna(0, inplace=True)
params1 = {
    'n_estimators': [100, 300],
    'max_depth': [10, 50, 100],
    'min_samples_split': [2, 10, 20]
}
model1 = RandomForestRegressor()
model1 = GridSearchCV(model1, params1, cv=6)

   year  price  mileage    tax   mpg  engineSize  tax(£)  transmission_Manual  \
0  2018   7885     9876  145.0  55.4         1.4     NaN                 True   
1  2019  11995     2500  145.0  54.3         1.4     NaN                 True   
2  2017   9777     9625  145.0  47.9         1.4     NaN                False   
3  2016   8500    25796   30.0  55.4         1.4     NaN                 True   
4  2019  10000     3887  145.0  43.5         1.4     NaN                 True   

   transmission_Other  transmission_Semi-Auto  ...  model_ Z3  model_ Z4  \
0               False                   False  ...      False      False   
1               False                   False  ...      False      False   
2               False                   False  ...      False      False   
3               False                   False  ...      False      False   
4               False                   False  ...      False      False   

   model_ Zafira  model_ Zafira Tourer  model_ i3  model

In [140]:
model1.fit(x_train, y_train)

GridSearchCV(cv=6, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 50, 100],
                         'min_samples_split': [2, 10, 20],
                         'n_estimators': [100, 300]})

In [141]:
print(model1.best_params_)

{'max_depth': 100, 'min_samples_split': 10, 'n_estimators': 100}


In [142]:
predict1 = model1.best_estimator_.predict(x_test)

In [143]:
print(r2_score(y_test, predict1))

0.9528374993828238


In [157]:
from sklearn.tree import DecisionTreeRegressor

params2 = {
    'max_depth': [10, 20, 30, 40],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 'log2']
}
model2 = DecisionTreeRegressor()
model2 = GridSearchCV(model2, params2, cv=6)

In [158]:
model2.fit(x_train, y_train)

GridSearchCV(cv=6, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [10, 20, 30, 40],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 5, 10]})

In [159]:
print(model2.best_params_)

{'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 1}


In [160]:
predict2 = model2.best_estimator_.predict(x_test)

In [161]:
print(r2_score(y_test, predict2))

0.9124069332186888


##### Итоги
1) DecisionTree обучается сильно быстрее (примерно за 2 минуты), с другой стороны RandomForest обучался почти час
2) RandomForest дал лучшую точность, однако нельзя сказать, что при выбранных мной параметрах разница критична
3) Однако, приведу пример, когда при других параметрах DecisionTree дает более низкую точность

In [174]:
params3 = {
    'max_depth': [10, 13, 16, 19],
    'min_samples_split': [2, 3, 4],
    'max_features': ['sqrt', 'log2']
}
model3 = DecisionTreeRegressor()
model3 = GridSearchCV(model3, params3, cv=6)

In [175]:
model3.fit(x_train, y_train)

GridSearchCV(cv=6, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [10, 13, 16, 19],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 3, 4]})

In [176]:
print(model3.best_params_)

{'max_depth': 19, 'max_features': 'sqrt', 'min_samples_split': 2}


In [177]:
predict3 = model3.best_estimator_.predict(x_test)

In [178]:
print(r2_score(y_test, predict3))

0.8551609625312167


Видим, что при менее удачных параметрах точность будет ниже